In [ ]:
import ee
from modules import abd

# Initialize Google Earth Engine
ee.Initialize()

# folder where to save results
folder = "/path/to/desired/folder"

# create algorithm object
algorithm = abd.Abd(lat_lon="-83.50124371805877,41.88435023280987,-83.07548096199702,41.65275061592091,
                    date="2019-06-03",
                    days_threshold=180,
                    model="ocsvm",
                    sensor="modis")

# creating timeseries based on days_thresholds
algorithm.process_timeseries_data()

# creating train and grid datasets
algorithm.process_training_data(df=algorithm.df_timeseries)

# start training process
algorithm.train()

# apply detection algorithm
algorithm.detect()

# validate using ROI - example (https://code.earthengine.google.com/)
algorithm.validate_using_roi(path='users/gee-user-name/erie', rois=['2019-06-03_modis_regular', '2019-06-03_modis_anomaly'], labels=[0, 1])

# save geojson
algorithm.save_geojsons(folder=folder+"/geojson")

# save results
algorithm.save_dataset(df=algorithm.df_results, path=folder+'/results.csv')

# save collection images
algorithm.save_timeseries_plot(df=algorithm.df_timeseries, path=folder+'/timeseries.png')
if isinstance(algorithm.df_image, pd.DataFrame) and not algorithm.df_image.empty:
    algorithm.save_detection_plot(path=folder+'/detection.png')
    algorithm.save_image_png(image=algorithm.image_clip, date=dt.strptime(date, "%Y-%m-%d"), path=folder+"/"+date+".png")
    algorithm.save_image_tiff(image=algorithm.image_clip, date=dt.strptime(date, "%Y-%m-%d"), path=folder+"/"+date+".zip", folderName=args.name)
    
# save collection images (tiff and png)
if args.save_collection:
    algorithm.save_collection_tiff(folder=folder+"/tiff", folderName="abd_erie.tiff")
    algorithm.save_collection_png(folder=folder+"/png")

# save preprocessing results
if args.save_train:
    algorithm.save_dataset(df=algorithm.df_timeseries, path=folder+'/timeseries.csv')
    algorithm.save_dataset(df=algorithm.df_train, path=folder+'/df_train.csv')
    algorithm.save_dataset(df=algorithm.df_gridsearch, path=folder+'/df_gridsearch.csv')
    algorithm.save_dataset(df=algorithm.df_image, path=folder+'/df_image.csv')